In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import time

# Directions for use:
    
1. Specify benthic conditions **(AltSS, MacroDom, CoralDom)** in code block 3
2. Use **AltSS_finder function** to add one, two, or three fish to system
3. Run the **AltSS_finder function** at three levels of η (0, 0.5, 1)

## ODE System

In [22]:
def dNdt(t, N, rC, rM, rT, αCT, αMT, αMC, μC, μM, μT, #benthos
         hB, ηB, sB, σB, θB, eB, pB,                  #browsers
         hG, ηG, sG, σG, θG, eG, pG,                  #grazers
         hS, ηS, sS, σS, θS, eS, pS):                 #scrapers
    
    C,M,T,B,G,S = N
    
    dC = (rC*C*(1-C-M-T) + αCT*rC*C*T - αMC*rM*C*M - μC*C)*dt
    dM = (rM*M*(1-C-M-T) + αMT*rM*T*M + αMC*rM*C*M - μM*M - ((hB*B*M)/((hB*ηB*M)+1)))*dt
    dT = (rT*T*(1-C-M-T) - αCT*rC*C*T - αMT*rM*T*M - μT*T - ((hG*G*T)/((hG*ηG*T)+1)) - ((hS*S*T)/((hS*ηS*T)+1)))*dt
    
    dB = (sB*B*(1-(B/((1-σB)+(σB*C)))) + θB*B - eB*B - pB*B)*dt
    dG = (sG*G*(1-(G/((1-σG)+(σG*C)))) + θG*G - eG*G - pG*G)*dt
    dS = (sS*S*(1-(S/((1-σS)+(σS*C)))) + θS*S - eS*S - pS*S)*dt

    return dC, dM, dT, dB, dG, dS

## Initializing Benthos and Timesteps

1. In these runs, the initial benthic conditions promote AltSS.

In [23]:
#μM_MacroDom_11 = 0.11
μM_AltSS_12 = 0.12
#μM_CoralDom_13 = 0.13

#Macroalgae Dominant------------------------------
#rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_MacroDom_11, 10

#Alternative Stable States------------------------
rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_AltSS_12, 10

#Coral Dominant-----------------------------------
#rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_CoralDom_13, 10


In [24]:
dt = 1
NUMSTEPS = 5000 #you should eventually add a term that terminates solving after reaching equilibrium
NUMYEARS = int(NUMSTEPS/dt)
time_points = np.linspace(0, NUMYEARS, NUMSTEPS+1)

h_val = np.linspace(0,1,11)
rM_val = np.linspace(0,1,11)

## Function to find benthic state at increasing rates of h (herbivory rate) and rM (macroalgae growth) at three levels of n (palatability)

In [130]:
def AltSS_finder(scenario, η): 
    
    outputs = []
    outputs_prime = []
    
    for h in h_val:
        #print ('h='+str(h))

        for rM in rM_val:
            #print ('rM='+str(rM))
        
            for C0 in np.linspace(0.01,1,5): #you changed this to 5
                #print ('C0='+str(C0))
    
                for M0 in np.linspace(0.01,1,5):
    
                    for T0 in np.linspace(0.01,1,5):
    
                        if (C0 + M0 + T0 <= 1):

                            ηB, ηG, ηS =  η, η, η

                            #print (η)
        
                            #Baseline Benthos (No Herbivores)------------------------------
                            if scenario == 'baselinebenthos':
                                hB, ηB, sB, σB, θB, eB, pB = 0, ηB, 0, 0, 0, 0, 0
                                hG, ηG, sG, σG, θG, eG, pG = 0, ηG, 0, 0, 0, 0, 0
                                hS, ηS, sS, σS, θS, eS, pS = 0, ηS, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Browsers-----------------------------------------------------
                            elif scenario == 'browser':
                                hB, ηB, sB, σB, θB, eB, pB = h, ηB, 0.1, 1, 0, 0, 0.05
                                hG, ηG, sG, σG, θG, eG, pG = 0, ηG, 0, 0, 0, 0, 0
                                hS, ηS, sS, σS, θS, eS, pS = 0, ηS, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Grazers-----------------------------------------------------
                            elif scenario == 'grazer':
                                hG, ηG, sG, σG, θG, eG, pG = h, ηG, 0.2, 1, 0, 0, 0.05
                                hB, ηB, sB, σB, θB, eB, pB = 0, ηB, 0, 0, 0, 0, 0
                                hS, ηS, sS, σS, θS, eS, pS = 0, ηS, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Scrapers-----------------------------------------------------
                            elif scenario == 'scraper':
                                hS, ηS, sS, σS, θS, eS, pS = h, ηS, 0.5, 1, 0, 0, 0.05
                                hB, ηB, sB, σB, θB, eB, pB = 0, ηB, 0, 0, 0, 0, 0
                                hG, ηG, sG, σG, θG, eG, pG = 0, ηG, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Browsers & Grazers-------------------------------------------
                            elif scenario == 'browser_grazer':
                                hB, ηB, sB, σB, θB, eB, pB = h, ηB, 0.1, 1, 0, 0, 0.05
                                hG, ηG, sG, σG, θG, eG, pG = h, ηG, 0.2, 1, 0, 0, 0.05
                                hS, ηS, sS, σS, θS, eS, pS = 0, ηS, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Browsers & Scrapers------------------------------------------
                            elif scenario == 'browser_scraper':
                                hB, ηB, sB, σB, θB, eB, pB = h, ηB, 0.1, 1, 0, 0, 0.05
                                hS, ηS, sS, σS, θS, eS, pS = h, ηS, 0.5, 1, 0, 0, 0.05
                                hG, ηG, sG, σG, θG, eG, pG = 0, ηG, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Grazers & Scrapers------------------------------------------
                            elif scenario == 'grazer_scraper':
                                hG, ηG, sG, σG, θG, eG, pG = h, ηG, 0.2, 1, 0, 0, 0.05
                                hS, ηS, sS, σS, θS, eS, pS = h, ηS, 0.5, 1, 0, 0, 0.05
                                hB, ηB, sB, σB, θB, eB, pB = 0, ηB, 0, 0, 0, 0, 0
                                rM = rM
    
                            #Browsers, Grazers, & Scrapers--------------------------------
                            elif scenario == 'all_herbivores':
                                hB, ηB, sB, σB, θB, eB, pB = h, ηB, 0.1, 1, 0, 0, 0.05
                                hG, ηG, sG, σG, θG, eG, pG = h, ηG, 0.2, 1, 0, 0, 0.05
                                hS, ηS, sS, σS, θS, eS, pS = h, ηS, 0.5, 1, 0, 0, 0.05
                                rM = rM
                            
                            sol = solve_ivp(dNdt, [0,NUMYEARS], [C0, M0, T0, B0, G0, S0], 
                                            method = 'RK45', args = 
                                            (rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                              #benthos
                                            hB, ηB, sB, σB, θB, eB, pB,                                          #browsers
                                            hG, ηG, sG, σG, θG, eG, pG,                                          #grazers
                                            hS, ηS, sS, σS, θS, eS, pS),                                         #scrapers
                                            dense_output=True)
    
                            N = sol.sol(time_points)
    
                            C_array = N[0,:]
                            M_array = N[1,:]
                            T_array = N[2,:]
                            B_array = N[3,:]
                            G_array = N[4,:]
                            S_array = N[5,:]

                            
                            F_prime = 1-(C_array[-1] + M_array[-1] + T_array[-1])

                            if C_array[-1] >= 0.01 and M_array[-1] < 0.01 and T_array[-1] < 0.01:
                                color_col = 'allC'
                            
                            elif M_array[-1] >= 0.01 and C_array[-1] < 0.01 and T_array[-1] < 0.01:
                                color_col = 'allM'
                            
                            elif T_array[-1] >= 0.01 and C_array[-1] < 0.01 and M_array[-1] < 0.01:
                                color_col = 'allT'
                            
                            elif C_array[-1] < 0.01 and M_array[-1] < 0.01 and T_array[-1] < 0.01:
                                color_col = 'allF'
                                
                            elif C_array[-1] >= 0.01 and M_array[-1] >= 0.01 and T_array[-1] >= 0.01 and C_array[-1] > M_array[-1] and C_array[-1] > T_array[-1] and C_array[-1] > F_prime:
                                color_col = 'mostlyC'
                            
                            elif C_array[-1] >= 0.01 and M_array[-1] >= 0.01 and T_array[-1] < 0.01 and C_array[-1] > M_array[-1] and C_array[-1] > T_array[-1] and C_array[-1] > F_prime:
                                color_col = 'mostlyC_noT'
                            
                            elif M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] >= 0.01 and M_array[-1] > C_array[-1] and M_array[-1] > T_array[-1] and M_array[-1] > F_prime:
                                color_col = 'mostlyM'
                            
                            elif M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] < 0.01 and M_array[-1] > C_array[-1] and M_array[-1] > T_array[-1] and M_array[-1] > F_prime:
                                color_col = 'mostlyM_noT'
                            
                            elif T_array[-1] >= 0.01 and M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] > C_array[-1] and T_array[-1] > M_array[-1] and T_array[-1] > F_prime:
                                color_col = 'mostlyT'

                            else:
                                color_col = 'other'
    
                            outputs.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                              #benthos
                                            hB, ηB, sB, σB, θB, eB, pB,                                         #browsers
                                            hG, ηG, sG, σG, θG, eG, pG,                                         #grazers
                                            hS, ηS, sS, σS, θS, eS, pS,                                         #scrapers
                                            C_array, M_array, T_array, B_array, G_array, S_array,
                                            color_col))
                                
                            outputs_prime.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                        #benthos
                                            hB, ηB, sB, σB, θB, eB, pB,                                         #browsers
                                            hG, ηG, sG, σG, θG, eG, pG,                                         #grazers
                                            hS, ηS, sS, σS, θS, eS, pS,                                         #scrapers
                                            C_array[-1], M_array[-1], T_array[-1], F_prime, B_array[-1], G_array[-1], S_array[-1],
                                            color_col))
    
    df_timeseries = pd.DataFrame(outputs, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT', #benthos
                                            'hB', 'ηB', 'sB', 'σB', 'θB', 'eB', 'pB',                           #browsers
                                            'hG', 'ηG', 'sG', 'σG', 'θG', 'eG', 'pG',                           #grazers
                                            'hS', 'ηS', 'sS', 'σS', 'θS', 'eS', 'pS',                           #scrapers
                                            'C_array', 'M_array', 'T_array', 'B_array', 'G_array', 'S_array',
                                            'color_col'])
        
        
    df_prime = pd.DataFrame(outputs_prime, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT', #benthos
                                            'hB', 'ηB', 'sB', 'σB', 'θB', 'eB', 'pB',                            #browsers
                                            'hG', 'ηG', 'sG', 'σG', 'θG', 'eG', 'pG',                            #grazers
                                            'hS', 'ηS', 'sS', 'σS', 'θS', 'eS', 'pS',                            #scrapers
                                            'C_prime', 'M_prime', 'T_prime', 'F_prime','B_prime', 'G_prime', 'S_prime', 
                                            'color_col'])
        
    return(df_timeseries, df_prime)

# η = 0

## Running Function
1. Specify which **scenario (i.e. which herbivores are included)** and **save as pickle file**

In [95]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browsers_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'browser', η = 0)

Browsers_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_Browsers_AltSS_μM12_hrM_η0_030825')
Browsers_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_Browsers_AltSS_μM12_hrM_η0_030825')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'grazer', η = 0)

Grazer_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_Grazer_AltSS_μM12_hrM_η0_030825')
Grazer_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_Grazer_AltSS_μM12_hrM_η0_030825')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'scraper', η = 0)

Scraper_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_Scraper_AltSS_hrM_η0_030825')
Scraper_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_Scraper_AltSS_hrM_η0_030825')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'browser_grazer', η = 0)

BrowserGrazer_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_BrowserGrazer_AltSS_μM12_hrM_η0_030825')
BrowserGrazer_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_BrowserGrazer_AltSS_μM12_hrM_η0_030825')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'browser_scraper', η = 0)

BrowserScraper_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_BrowserScraper_AltSS_μM12_hrM_η0_030825')
BrowserScraper_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_BrowserScraper_AltSS_μM12_hrM_η0_030825')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'grazer_scraper', η = 0)

GrazerScraper_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_GrazerScraper_AltSS_μM12_hrM_η0_030825')
GrazerScraper_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_GrazerScraper_AltSS_μM12_hrM_η0_030825')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'all_herbivores', η = 0)

AllHerbivores_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_AllHerbivores_AltSS_μM12_hrM_η0_030825')
AllHerbivores_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_AllHerbivores_AltSS_μM12_hrM_η0_030825')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_AltSS_μM12_hrM_η0_030825  = AltSS_finder(scenario = 'baselinebenthos', η = 0)

BaselineBenthos_AltSS_μM12_hrM_η0_030825[0].to_pickle('timeseries_BaselineBenthos_AltSS_μM12_hrM_η0_030825')
BaselineBenthos_AltSS_μM12_hrM_η0_030825[1].to_pickle('equilibria_BaselineBenthos_AltSS_μM12_hrM_η0_030825')


h=0.0
rM=0.0


KeyboardInterrupt: 

# η = 0.5

## Running Function
1. Specify which **scenario (i.e. which herbivores are included)** and **save as pickle file**

In [ ]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browsers_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'browser', η = 0.5)

Browsers_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_Browsers_AltSS_μM12_hrM_η05_030825')
Browsers_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_Browsers_AltSS_μM12_hrM_η05_030825')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'grazer', η = 0.5)

Grazer_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_Grazer_AltSS_μM12_hrM_η05_030825')
Grazer_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_Grazer_AltSS_μM12_hrM_η05_030825')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'scraper', η = 0.5)

Scraper_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_Scraper_AltSS_hrM_η05_030825')
Scraper_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_Scraper_AltSS_hrM_η05_030825')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'browser_grazer', η = 0.5)

BrowserGrazer_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_BrowserGrazer_AltSS_μM12_hrM_η05_030825')
BrowserGrazer_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_BrowserGrazer_AltSS_μM12_hrM_η05_030825')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'browser_scraper', η = 0.5)

BrowserScraper_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_BrowserScraper_AltSS_μM12_hrM_η05_030825')
BrowserScraper_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_BrowserScraper_AltSS_μM12_hrM_η05_030825')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'grazer_scraper', η = 0.5)

GrazerScraper_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_GrazerScraper_AltSS_μM12_hrM_η05_030825')
GrazerScraper_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_GrazerScraper_AltSS_μM12_hrM_η05_030825')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'all_herbivores', η = 0.5)

AllHerbivores_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_AllHerbivores_AltSS_μM12_hrM_η05_030825')
AllHerbivores_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_AllHerbivores_AltSS_μM12_hrM_η05_030825')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_AltSS_μM12_hrM_η05_030825  = AltSS_finder(scenario = 'baselinebenthos', η = 0.5)

BaselineBenthos_AltSS_μM12_hrM_η05_030825[0].to_pickle('timeseries_BaselineBenthos_AltSS_μM12_hrM_η05_030825')
BaselineBenthos_AltSS_μM12_hrM_η05_030825[1].to_pickle('equilibria_BaselineBenthos_AltSS_μM12_hrM_η05_030825')


# η = 1

## Running Function
1. Specify which **scenario (i.e. which herbivores are included)** and **save as pickle file**

In [ ]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browsers_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'browser', η = 1)

Browsers_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_Browsers_AltSS_μM12_hrM_η1_030825')
Browsers_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_Browsers_AltSS_μM12_hrM_η1_030825')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'grazer', η = 1)

Grazer_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_Grazer_AltSS_μM12_hrM_η1_030825')
Grazer_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_Grazer_AltSS_μM12_hrM_η1_030825')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'scraper', η = 1)

Scraper_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_Scraper_AltSS_hrM_η1_030825')
Scraper_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_Scraper_AltSS_hrM_η1_030825')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'browser_grazer', η = 1)

BrowserGrazer_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_BrowserGrazer_AltSS_μM12_hrM_η1_030825')
BrowserGrazer_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_BrowserGrazer_AltSS_μM12_hrM_η1_030825')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'browser_scraper', η = 1)

BrowserScraper_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_BrowserScraper_AltSS_μM12_hrM_η1_030825')
BrowserScraper_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_BrowserScraper_AltSS_μM12_hrM_η1_030825')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'grazer_scraper', η = 1)

GrazerScraper_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_GrazerScraper_AltSS_μM12_hrM_η1_030825')
GrazerScraper_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_GrazerScraper_AltSS_μM12_hrM_η1_030825')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'all_herbivores', η = 1)

AllHerbivores_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_AllHerbivores_AltSS_μM12_hrM_η1_030825')
AllHerbivores_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_AllHerbivores_AltSS_μM12_hrM_η1_030825')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_AltSS_μM12_hrM_η1_030825  = AltSS_finder(scenario = 'baselinebenthos', η = 1)

BaselineBenthos_AltSS_μM12_hrM_η1_030825[0].to_pickle('timeseries_BaselineBenthos_AltSS_μM12_hrM_η1_030825')
BaselineBenthos_AltSS_μM12_hrM_η1_030825[1].to_pickle('equilibria_BaselineBenthos_AltSS_μM12_hrM_η1_030825')
